In [1]:
import json
import pandas as pd
from tqdm import tqdm_notebook

In [ ]:
with open('/home/wowns/data/Crawling/data_info.json', "r") as json_file:
        json_data = json.load(json_file)

title = []
num = []
context = []
count = 0
for i in json_data['data']:
    title.append(i['title'])
    num.append(i['count'])
    
data = {
    'title':title,
    'num': num,
}
df = pd.DataFrame(data)
sorted = df.sort_values(by='num' ,ascending=False)
sorted.to_csv('sorted_title_by_num.csv',index=False)

In [ ]:
from util.func import remove_bracket
title_sort = pd.read_csv('/home/hsoh0423/vscode/HistoryQA/sorted_title_by_num.csv')
title_rm_list = []
title_rm_count = []
for i in range(len(title_sort)):
    try:
        title_rm = remove_bracket(title_sort['title'][i])
        title_rm_list.append(title_rm)
        title_rm_count.append(title_sort['num'][i])
    except:
        continue
    
data = {
    'title': title_rm_list,
    'num': title_rm_count
}
df = pd.DataFrame(data)
df.to_csv('sorted_title_rm.csv',index=False)


In [ ]:
title_sort = pd.read_csv('/home/hsoh0423/vscode/HistoryQA/sorted_title_rm.csv')

title2 = []
title_count2 = []


count = 0
for i in tqdm_notebook(range(len(title_sort))):
    
    if  title_sort['title'][i] in title_list and title_sort['num'][i] > 150:
        title2.append(title_sort['title'][i])
        title_count2.append(title_sort['num'][i])

data = {
    'title': title2,
    'num': title_count2
}
df = pd.DataFrame(data)
df.to_csv('sorted_title_preQuiz.csv',index=False)

In [2]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import sys
from os import path
import numpy as np
from tqdm import tqdm
from gensim.models import Word2Vec
from Embeder import SentenceEmbeder
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
if __package__ is None:
    current_path = path.dirname(path.dirname(path.abspath(
        '/home/fhdufhdu/vscode/KoreanHistoryProject/keyword_extractor/keyword_extractor_ver_W2V.ipynb')))
    sys.path.append(current_path)
    from func import save_pickle, load_pickle, get_tokenized_sentences, remove_duplicated_keyword, save_json
else:
    from func import save_pickle, load_pickle, get_tokenized_sentences, remove_duplicated_keyword, save_json


title_list, sent_list = get_tokenized_sentences('.', True)
title_doc_list = load_pickle('/home/fhdufhdu/vscode/Project/data/keyword_extractor_data/title_doc_list.pickle')


# embedding_model = Word2Vec(sent_list, size=100, window = 5, min_count=1, workers=4, iter=50, sg=1)
# save_pickle('w2v_model/w2v.pickle', embedding_model)

w2v_model = load_pickle('/home/fhdufhdu/vscode/Project/KoreanHistoryProject/keyword_extractor/w2v_model/w2v.pickle')
bert_embeder = SentenceEmbeder(10)

using cached model


[transformers.configuration_utils] loading configuration file /home/hsoh0423/kobert/kobert_from_pretrained/config.json
[transformers.configuration_utils] Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 2,
  "vocab_size": 8002
}

[transformers.modeling_utils] loading weights file /home/hsoh0423/kobert/kobert_from_pretrained/pytorch_model.bin


using cached model
using cached model


In [4]:
def get_keyword(title, words, w2v, bert):
    # bert
    _, title_vector = bert.encode([title])
    _, noun_vectors = bert.encode(words)
    title_vector = title_vector[0]


    title_embedding_ = [[value.item() for value in title_vector]]
    candicates_embedding_ = [[value.item() for value in noun_vector]
                            for noun_vector in noun_vectors]
    distances = cosine_similarity(title_embedding_, candicates_embedding_)
    b_similar_list = [(words[i], sim) for i, sim in enumerate(distances[0])]

    # word2vec
    w_similar_list = []
    for word in words:
        try:
            w_similar_list.append((word, w2v.wv.similarity(w1=title_list[0], w2=word)))
        except:
            continue

    final_list = []
    for (w_t, w_s), (_, b_s) in zip(w_similar_list, b_similar_list):
        final_list.append((w_t, w_s + b_s))

    final_list = sorted(final_list, key=lambda sim: sim[1], reverse=True)

    return final_list

In [5]:
quiz_title_list = pd.read_csv('/home/hsoh0423/vscode/HistoryQA/test_title.csv')
quiz_title_list2 = []
for i in quiz_title_list['title']:
    quiz_title_list2.append(i)

qas_dict = {}
qas_dict['data'] = []
count = 0
for i in tqdm_notebook(quiz_title_list2):
    try:
        doc_idx = 0
        sent_idx = 0
        count += 1
        while sent_idx < len(title_doc_list):
            title = title_list[sent_idx]
            title_with_doc, doc = title_doc_list[sent_idx]
            sent = sent_list[sent_idx]

            if i == title:
                qas_set = {}
                qas_set['title'] = title
                qas_set['doc'] = doc
                qas_set['answer'] = title
                qas_dict['data'].append(qas_set)
                if title == title_with_doc:

                        keywords = get_keyword(title, remove_duplicated_keyword(sent), w2v_model, bert_embeder)[:5]
                        for keyword, _ in keywords:
                            qas_set = {}
                            qas_set['title'] = title
                            qas_set['doc'] = doc
                            qas_set['answer'] = keyword
                            qas_dict['data'].append(qas_set)

                if count % 100 == 0 :
                    print("save as", count)
                    save_json('as_set_V2.json', qas_dict)
            sent_idx += 1  
    except:
        print('catch exception')
        continue
        
save_json('as_set_V2.json', qas_dict)


/home/hsoh0423/anaconda3/envs/historyQA/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/7796 [00:00<?, ?it/s]

save as 100
save as 200
save as 300
save as 300
save as 400
save as 400
catch exception
save as 500
save as 600
save as 700
save as 700
save as 700
save as 800
save as 900
save as 900
save as 1000
save as 1000
save as 1100
save as 1200
save as 1300
save as 1400
save as 1500
save as 1600
save as 1700
save as 1800
save as 1800
save as 1800
save as 1900
save as 1900
save as 2000
save as 2100
save as 2100
save as 2200
save as 2300
save as 2400
save as 2500
save as 2500
save as 2600
save as 2700
save as 2700
save as 2800
save as 2900
save as 3000
save as 3100
save as 3100
save as 3200
save as 3300
save as 3300
save as 3400
save as 3400
save as 3400
save as 3400
save as 3500
save as 3600
save as 3700
save as 3800
save as 3800
save as 3900
save as 3900
save as 4000
save as 4100
save as 4200
save as 4300
save as 4400
save as 4500
save as 4600
save as 4700
save as 4800
save as 4800
save as 4900
save as 5000
save as 5100
save as 5200
save as 5200
save as 5300
save as 5400
save as 5500
save as 56

In [ ]:
with open('/home/hsoh0423/vscode/HistoryQA/as_set_test.json', "r") as json_file:
        json_data = json.load(json_file)

title_list_count = []
for i in json_data['data']:
        title_list_count.append(i['title'])
        
title_list_count2 = list(set(title_list_count))
print(len(title_list_count2))
for i in quiz_title_list2[:471]:
        if i not in title_list_count2:
                print(i)


In [ ]:
save_json('as_set_test.json', qas_dict)

In [ ]:
# %%
qas_dict = {}
qas_dict['data'] = []
doc_idx = 0
sent_idx = 0
while sent_idx < 10 and doc_idx < len(title_doc_list):
    title = title_list[sent_idx]
    title_with_doc, doc = title_doc_list[doc_idx]
    sent = sent_list[sent_idx]
    if title != title_with_doc:
        doc_idx += 1
        continue
    print(title, title_with_doc)
    keywords = get_keyword(title, remove_duplicated_keyword(
        sent), w2v_model, bert_embeder)[:5]
    for keyword, _ in keywords:
        qas_set = {}
        qas_set['doc'] = doc
        qas_set['answer'] = keyword
        qas_dict['data'].append(qas_set)
    qas_set = {}
    qas_set['doc'] = doc
    qas_set['answer'] = [title]
    qas_dict['data'].append(qas_set)
    sent_idx += 1
    print('progress : {}/{}'.format(sent_idx, 5000))

save_json('as_set_test.json', qas_dict)